# Automatic Data Type Discovery

In [1]:
from scipy.io import savemat
import openml as oml
import pandas as pd
import numpy as np
import random

## 1. Heuristic Methods

In [2]:
def infer_feature_type(feature):
    """
    Infer data types for the given feature using simple logic
    Possible data types to infer: boolean, date, float, integer, string
    Feature that is not either a boolean, a date, a float or an integer, is classified as a string
    """
    data_type = ""
    types = ["date","float64","int64","string"]
    weights = [0,0,0,0] #Weights corresponding to the data types
    feature_len = len(feature)
    
    indices_number = int(0.1 * feature_len) #Number of different values to check in a feature
    indices = random.sample(range(0,feature_len), min(indices_number,feature_len)) #Array of random indices
    
    # If the feature only contains two different unique values, then infer it as boolean
    if len(pd.unique(feature)) == 2:
        data_type = "bool"
    else:
        for i in indices:
            try:
                if (len(feature[i]) <= 10 
                    and (((feature[i][2:3]=='-' or feature[i][2:3]=='/') 
                    and (feature[i][5:6]=='-' or feature[i][5:6]=='/')) 
                    or ((feature[i][4:5]=='-' or feature[i][4:5]=='/')
                    and (feature[i][7:8]=='-' or feature[i][7:8]=='/')))):
                    weights[0] += 1 #Date
                else:
                    weights[3] += 1 #String
            except (TypeError,ValueError,IndexError):
                try:
                    int(feature[i]) # numeric
                    if ('.' in str(feature[i])):
                        if isinstance(feature[i], np.float64):
                            if feature[i].is_integer():
                                weights[2] += 1 # Integer
                            else:
                                weights[1] += 1 # Float
                        else:
                            weights[1] += 1 # Float
                    else:
                        weights[2] += 1 #Integer
                except (TypeError,ValueError,IndexError):
                    weights[3] += 1 #String
#         print ("Date: {}, Float64: {}, Int64: {}, String: {}".format(weights[0],weights[1],weights[2],weights[3])) #For debugging purposes
        data_type = types[weights.index(max(weights))]
        
    return data_type

def discover_type_heuristic(data):
    """
    Automatically infer data types for each feature in the dataframe using simple logic
    ---
    input:

    data : numpy array or dataframe (numeric data needs to be 64 bit)
        
    """
#     df = pd.DataFrame(data)
#     print(df)
    result = []
    if isinstance(data, np.ndarray):
        # convert float32 to float64
        data = np.array(data, dtype='float64')
        df = pd.DataFrame(data)
    else:
        df = data

    for column in df.columns:        
        #print ("Trying to automatically infer the data type of the",column,"feature...") #For debugging purposes
        type_inferred = infer_feature_type(df[column])
        result.append(type_inferred)
        #print ("Result:",inferredType) #For debugging purposes
    return result

### 1.1 Get data from csv

In [3]:
df = pd.read_csv("datasets_csv/tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
discover_type_heuristic(df)

['float64', 'float64', 'bool', 'bool', 'string', 'bool', 'int64']

### 1.2 Get data from openml

In [5]:
data = oml.datasets.get_dataset(1480)
Xy = data.get_data()
Xy.shape

(583, 11)

In [6]:
discover_type_heuristic(Xy)

['int64',
 'bool',
 'float64',
 'float64',
 'int64',
 'int64',
 'int64',
 'float64',
 'float64',
 'float64',
 'bool']

## 2. Bayesian Method
To apply bayesian methos, dataset needs to be parsed into tabular numeric form beforehand.

### Get data from openml

In [10]:
data = oml.datasets.get_dataset(40498)
Xy = data.get_data()
Xy.shape

(4898, 12)

### Generate mat file

In [11]:
def generate_mat(data, extra_cardinality=1):
    simple_types = discover_type_heuristic(data)
    # map simple types to meta types
    # 1: real (w positive: all real | positive | interval)
    # 2: real (w/o positive: all real | interval)
    # 3: binary data
    # 4: discrete (non-binary: categorical | ordinal | count)
    # note: for now, the implemented bayesian method version by isabel can distinguish between real, postive real, categorical and count
    # the binary data should be mapped to meta type 4 discrete instead of meta type 3 due to the limited implemented version. This may change
    # if the extended version has been implemented by isabel.
    meta_types = [] 
    for i in range(len(simple_types)):
#         print(simple_types[i])
        if simple_types[i] == "bool":
            meta_types.append(4) # may change in the future
        elif simple_types[i] == "int64" or simple_types[i] == "float64":
            if (len(np.unique(data[:,i])) < 0.02 * len(data[:,i]) and len(np.unique(data[:,i])) < 50):
                meta_types.append(4)
            else:
                if (data[:,i] > 0).all():
                    meta_types.append(1)
                else:
                    meta_types.append(2)
        else:
            meta_types.append(1)
    discrete_cardinality = [] # number of unique for discrete feature, 1 for others
    for i in range(len(meta_types)):
        if (meta_types[i] == 4):
            discrete_cardinality.append(int(np.max(data[:,i])) + extra_cardinality) 
        else:
            discrete_cardinality.append(1)
    data_dict = {'X': data,
                 'T': np.asarray(meta_types),
                 'R': np.asarray(discrete_cardinality)}
    print(data_dict)
    savemat('bayesian/data/temp/data.mat', data_dict, oned_as='row')

In [12]:
generate_mat(Xy)

{'X': array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  3.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  3.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  3.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  3.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  4.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  3.  ]], dtype=float32), 'T': array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7])}


### Call bayesian model

In [13]:
from bayesian.bin import abda

D:\Programfiles\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)


Loaded bayesian/data/temp/data.mat
	with shape: (4898, 12)
	meta-types:[1 1 2 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [1 1 1 1 1 1 1 1 1 1 1 7]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 3.79999995, 14.19999981]), array([0.08      , 1.10000002]), array([0.        , 1.65999997]), array([ 0.60000002, 65.80000305]), array([0.009     , 0.34599999]), array([  2., 289.]), array([  9., 440.]), array([0.98711002, 1.03898001]), array([2.72000003, 3.81999993]), array([0.22      , 1.08000004]), array([ 8.        , 14.19999981]), array([0, 1, 2, 3, 4, 5, 6], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussia

{'params': {'alpha': 1.9967129409128155, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE6149080>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE61492E8>}
gamma fit params (10.443556298462036, 0, 0.003776674490135308)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE6149860>}
{'params': {'alpha': 10.443556298462036, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE6149B00>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE61494A8>}
gamma fit params (4.007340933194108, 0, 7.1208967527828975)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE6149FD0>}
{'params': {'alpha': 4.007340933194108, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma o

 0.00149421])) + 0.3333333333333333*(Geometric(V11|p=0.5)) + 0.3333333333333333*(Poisson(V11|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Done iteration 1/1 in 0.26527409887060305


------------------
Done in 0.27849566360467515
Dumped id to ./exp/temp/data.mat\20181105-194903\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-194903\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-194903


In [13]:
weights

{0: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.001975280707022,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.998024719292978},
 1: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.2505448190105733,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7494551809894267},
 2: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.17809000957006083,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.821909990429939},
 3: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.2148417757517452,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7851582242482545},
 4: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.1705409631051393,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.8294590368948606},
 5: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.0013720122627796153,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9986279877372204},
 6: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.000991269639289043,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9990087303607109},
 7: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.4025493630293431,
  <Type.POSITIVE: (3, <MetaType.REAL: 

## 3. Evaluation
We evaluate the bayesian method on the 30 most runs openml datasets. The ids of the 30 openml datasets are as follows: <br>
31, 1464, 334, 50, 333, 1494, 3, 1510, 1489, 37, 1479, 1487, 1063, 1471, 1467, 1480, 1068, 1492, 1050, 1462, 1046, 1049, 335, 

In [15]:
def get_statistical_type(weights):
    """
    Retrieve the key with the higher value from 'weights'
    The retrieved key is the statisical type of the corresponding feature
    """
    statistical_types = []
    for i in range(len(weights)):
#         print(max(weights[i], key=weights[i].get))
        statistical_types.append(str(max(weights[i], key=weights[i].get)))
    return statistical_types

In [16]:
def statistical_type_accurracy(ground_truth, statistical_types):
    count = 0 # number of correct predictions
    for i in range(len(ground_truth)):
        if ground_truth[i] == statistical_types[i]:
            count += 1
    return count / len(ground_truth)

In [17]:
def evaluate_bayesian(dataset_id, ground_truth):
    data = oml.datasets.get_dataset(dataset_id)
    Xy = data.get_data()
    Xy.shape
    try:
        generate_mat(Xy)
        weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)
    except:
        generate_mat(Xy, 10)
        weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)
    statistical_types = get_statistical_type(weights)
    print(statistical_types)
    accuracy = statistical_type_accurracy(ground_truth, statistical_types)
    return accuracy

In [18]:
# 31
ground_truth = ['Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.POSITIVE',
                'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGOTICAL', 'Type.COUNT', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(31, ground_truth)

{'X': array([[ 0.,  6.,  4., ...,  1.,  0.,  0.],
       [ 1., 48.,  2., ...,  0.,  0.,  1.],
       [ 3., 12.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 3., 12.,  2., ...,  0.,  0.,  0.],
       [ 0., 45.,  2., ...,  1.,  0.,  1.],
       [ 1., 45.,  4., ...,  0.,  0.,  0.]], dtype=float32), 'T': array([4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([ 4,  1,  5, 11,  1,  5,  5,  5,  4,  3,  5,  4,  1,  3,  3,  5,  4,
        3,  2,  2,  2])}

Loaded bayesian/data/temp/data.mat
	with shape: (1000, 21)
	meta-types:[4 1 4 4 1 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [ 4  1  5 11  1  5  5  5  4  3  5  4  1  3  3  5  4  3  2  2  2]
	meta types [<MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.REAL: 1> <Me

{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE621C198>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE614D518>}




CATEGORICAL




1 [0. 0. 0. 0.]
{'params': {'p': array([0.14137214, 0.23492723, 0.3970894 , 0.22661123])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE614DA90>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE614DEB8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE614DB70>}
gamma fit params (11.487864480868597, 0, 3.091936763260361)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE3F6A2E8>}
{'params': {'alpha': 11.487864480868597, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F6A400>}
{'params': {'l': 2}, '

{'params': {'p': array([0.03853565, 0.35645472, 0.47206166, 0.13294798])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4C162E8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4C16400>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4C16198>}




CATEGORICAL




1 [0. 0. 0.]
{'params': {'p': array([0.9017341, 0.0327553, 0.0655106])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4AF0780>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4AF07B8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4AF0748>}




CATEGORICAL




1 [0. 0. 0. 0. 0.]
{'params': {'p': array([1.52215800e-01, 3.06358381e-01, 1.79190751e-01, 3.62235067e-01,
       1.92678227e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichl

 1.92678227e-10])) + 0.3333333333333333*(Geometric(V15|p=0.5)) + 0.3333333333333333*(Poisson(V15|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V16|p=[0.02890173 0.27745665 0.61657033 0.07707129])) + 0.3333333333333333*(Geometric(V16|p=0.5)) + 0.3333333333333333*(Poisson(V16|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V17|p=[8.45857418e-01 1.54142582e-01 1.92678227e-10])) + 0.3333333333333333*(Geometric(V17|p=0.5)) + 0.3333333333333333*(Poisson(V17|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V18|p=[0.74181118 0.25818882])) + 0.3333333333333333*(Geometric(V18|p=0.5)) + 0.3333333333333333*(Poisson(V18|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V19|p=[0.93834297 0.06165703])) + 0.3333333333333333*(Geometric(V19|p=0.5)) + 0.3333333333333333*(Poisson(V19|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V20|p=[0.75915222 0.240

In [19]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.6190476190476191


In [20]:
# 1464
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1464, ground_truth)

{'X': array([[2.00e+00, 5.00e+01, 1.25e+04, 9.80e+01, 1.00e+00],
       [0.00e+00, 1.30e+01, 3.25e+03, 2.80e+01, 1.00e+00],
       [1.00e+00, 1.60e+01, 4.00e+03, 3.50e+01, 1.00e+00],
       ...,
       [2.30e+01, 3.00e+00, 7.50e+02, 6.20e+01, 0.00e+00],
       [3.90e+01, 1.00e+00, 2.50e+02, 3.90e+01, 0.00e+00],
       [7.20e+01, 1.00e+00, 2.50e+02, 7.20e+01, 0.00e+00]], dtype=float32), 'T': array([2, 1, 1, 1, 4]), 'R': array([1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (748, 5)
	meta-types:[2 1 1 1 4]
	maximal discrete cardinality: [1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 0., 74.]), array([ 1., 50.]), array([  250., 12500.]), array([ 2., 98.]), array([0, 1], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gau

(0.5*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=3.2188686184485946;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=3.2188686184485946;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=5.020117630242249;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V4|p=[0.68181818 0.31818182])) + 0.3333333333333333*(Geometric(V4|p=0.5)) + 0.3333333333333333*(Poisson(V4|mean=2))))) + 0.5*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(

In [21]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [22]:
# 334
ground_truth = ['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(334, ground_truth)

{'X': array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 2., 2., ..., 2., 2., 1.],
       [0., 2., 2., ..., 2., 3., 0.],
       [0., 2., 2., ..., 2., 3., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4]), 'R': array([2, 3, 3, 2, 3, 4, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (601, 7)
	meta-types:[4 4 4 4 4 4 4]
	maximal discrete cardinality: [2 3 3 2 3 4 2]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([0, 1], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2, 3], dtype=int64), array([0, 1], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<clas

(0.5008319467554077*((TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V6|p=[3.32225913e-10 1.00000000e+00])) + 0.3333333333333333*(Geometric(V6|p=0.5)) + 0.3333333333333333*(Poisson(V6|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V0|p=[0.63455149 0.36544851])) + 0.3333333333333333*(Geometric(V0|p=0.5)) + 0.3333333333333333*(Poisson(V0|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V1|p=[0.33222591 0.33554817 0.33222591])) + 0.3333333333333333*(Geometric(V1|p=0.5)) + 0.3333333333333333*(Poisson(V1|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V2|p=[0.33887043 0.33887043 0.32225914])) + 0.3333333333333333*(Geometric(V2|p=0.5)) + 0.3333333333333333*(Poisson(V2|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V3|p=[0.50166113 0.49833887])) + 0.3333333333333333*(Geometric(V3|p=0.5)) + 0.3333333333333333*(Poisson(V3|mean=2))) * TypeMixtureUnconst

In [23]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.8571428571428571


In [24]:
# 50
ground_truth = ['Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(50, ground_truth)

{'X': array([[2., 2., 2., ..., 1., 1., 1.],
       [2., 2., 2., ..., 2., 1., 1.],
       [2., 2., 2., ..., 1., 2., 1.],
       ...,
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 1., 2., ..., 2., 2., 0.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (958, 10)
	meta-types:[4 4 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [3 3 3 3 3 3 3 3 3 2]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64)

{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4AB0F98>}




CATEGORICAL




1 [0. 0. 0.]
{'params': {'p': array([0.23493976, 0.30421687, 0.46084337])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE3F911D0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE3F91240>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F912B0>}




CATEGORICAL




1 [0. 0. 0.]
{'params': {'p': array([0.14457831, 0.57831325, 0.27710843])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE3F913C8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE3F91518>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F91588>}




CATEGORICAL




1 [0. 0. 0.]
{'params': {'p': array([0.23493976, 0.3042

In [25]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.0


In [26]:
# 333
ground_truth = ['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(333, ground_truth)

{'X': array([[1., 0., 0., ..., 0., 2., 0.],
       [1., 0., 0., ..., 0., 2., 1.],
       [1., 0., 0., ..., 2., 1., 0.],
       ...,
       [1., 2., 2., ..., 2., 2., 1.],
       [1., 2., 2., ..., 2., 3., 0.],
       [1., 2., 2., ..., 2., 3., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4]), 'R': array([2, 3, 3, 2, 3, 4, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (556, 7)
	meta-types:[4 4 4 4 4 4 4]
	maximal discrete cardinality: [2 3 3 2 3 4 2]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([0, 1], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1], dtype=int64), array([0, 1, 2], dtype=int64), array([0, 1, 2, 3], dtype=int64), array([0, 1], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<clas

(0.4892086330935252*((TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V6|p=[1.00000000e+00 3.67647059e-10])) + 0.3333333333333333*(Geometric(V6|p=0.5)) + 0.3333333333333333*(Poisson(V6|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V0|p=[0.49632353 0.50367647])) + 0.3333333333333333*(Geometric(V0|p=0.5)) + 0.3333333333333333*(Poisson(V0|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V1|p=[0.34558824 0.34191176 0.3125    ])) + 0.3333333333333333*(Geometric(V1|p=0.5)) + 0.3333333333333333*(Poisson(V1|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V2|p=[0.32352941 0.33455882 0.34191176])) + 0.3333333333333333*(Geometric(V2|p=0.5)) + 0.3333333333333333*(Poisson(V2|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V3|p=[0.51102941 0.48897059])) + 0.3333333333333333*(Geometric(V3|p=0.5)) + 0.3333333333333333*(Poisson(V3|mean=2))) * TypeMixtureUnconst

In [27]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.8571428571428571


In [28]:
# 1494
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 
                'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 
                'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.REAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 
                'Type.COUNTL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1494, ground_truth)

{'X': array([[3.919 , 2.6909, 0.    , ..., 0.    , 0.    , 1.    ],
       [4.17  , 2.1144, 0.    , ..., 0.    , 0.    , 1.    ],
       [3.932 , 3.2512, 0.    , ..., 0.    , 0.    , 1.    ],
       ...,
       [4.869 , 1.767 , 0.    , ..., 1.    , 0.    , 0.    ],
       [5.158 , 1.6914, 2.    , ..., 0.    , 1.    , 0.    ],
       [5.076 , 2.6588, 2.    , ..., 0.    , 2.    , 0.    ]],
      dtype=float32), 'T': array([1, 1, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 1, 2, 1, 2, 1, 1, 4, 4, 4, 1,
       4, 4, 4, 4, 1, 2, 4, 2, 1, 4, 4, 4, 4, 1, 1, 4, 1, 4, 4, 4]), 'R': array([  1,   1,  13,   4,  37,  14,  19,   1,  25,  13,  45,   1,   1,
         1,   1,   1,   1,   1,   3,   4,   4,   1, 148,   2,   2,   4,
         1,   1,   2,   1,   1,   9,  13,  19,   8,   1,   1,   9,   1,
         5,  28,   2])}

Loaded bayesian/data/temp/data.mat
	with shape: (1055, 42)
	meta-types:[1 1 4 4 4 4 4 2 4 4 4 2 1 2 1 2 1 1 4 4 4 1 4 4 4 4 1 2 4 2 1 4 4 4 4 1 1
 4 1 4 4 4]
	maximal discrete cardinality: [  1 

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


gamma fit params (261.98377952482775, 0, 0.01958256442501167)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE3F80F98>}
{'params': {'alpha': 261.98377952482775, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F80DD8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F80D68>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.69097222e-01, 2.60416668e-02, 2.20486111e-01, 1.85763888e-01,
       1.52777777e-01, 3.64583334e-02, 5.20833333e-02, 1.21527779e-02,
       1.73611112e-02, 8.68055570e-03, 5.20833349e-03, 1.73611111e-10,
       8.68055570e-03, 1.73611111e-10, 1.73611128e-03, 1.73611111e-10,
       1.73611128e-03, 1.73611111e-10, 1.73611128e-03])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE61CB3C8>}
{'params': {'p': 0.5}, 

       2.08768267e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4B12940>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4B12438>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B123C8>}




CATEGORICAL




1 [0. 0. 0. 0.]
{'params': {'p': array([9.79123173e-01, 1.87891442e-02, 2.08768288e-03, 2.08768267e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4B125C0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4B120B8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B12D68>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([8.03757823e-01, 8.55949891e-02, 7.72442585e-02, 1.25260961e-02,
 

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0BCC0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0BD68>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0BE80>}
gamma fit params (858.4060291534967, 0, 0.0011682112534109057)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B09048>}
{'params': {'alpha': 858.4060291534967, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B09080>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B09198>}
gamma fit params (3804.9161848525623, 0, 0.0002997352870800073)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000

...

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0F6A0>}
{'params': {'alpha': 398.52224924819643, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0F5C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0F828>}
gamma fit params (27.910037280238196, 0, 0.10700371979994998)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0FA90>}
{'params': {'alpha': 27.910037280238196, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0F940>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0FB00>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([3.03030301e-09, 9.99999997e-01])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE

{'params': {'p': array([0.68100358, 0.31899642])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0EF8550>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0EF86A0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF8668>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([9.39068098e-01, 1.43369179e-02, 1.79211473e-02, 3.58422938e-10,
       1.43369179e-02, 3.58422938e-10, 1.07526885e-02, 3.58422938e-10,
       3.58422974e-03])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0EF89B0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0EF8860>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF8A20>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.90

       3.36700320e-10, 3.36700320e-10, 3.36700320e-10, 3.36700320e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0F1F208>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F1F390>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F1F400>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.86868687, 0.13131313])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0F1F438>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F1F6A0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F1F710>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([8.88888887e-01, 4.71380473e-02, 4.04040406e-02, 6.73400705e-03,
       1.34680138e-02, 3.36700369e-03, 3.36700336e-10, 3.36700336e-10,
       3.3670

       3.05810394e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0F0A080>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F0A278>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0A400>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([9.99999987e-01, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.05810393e-10,
       3.05810393e-10, 3.05810393e-10, 3.05810393e-10, 3.058

 2.08768266e-10 2.08768266e-10 2.08768266e-10 2.08768266e-10])) + 0.3333333333333333*(Geometric(V40|p=0.5)) + 0.3333333333333333*(Poisson(V40|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V41|p=[0.43215031 0.56784969])) + 0.3333333333333333*(Geometric(V41|p=0.5)) + 0.3333333333333333*(Poisson(V41|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
Done iteration 1/1 in 0.38642947090845325


------------------
Done in 0.3952972710513549
Dumped id to ./exp/temp/data.mat\20181105-194957\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-194957\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-194957
['Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.

In [29]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5952380952380952


In [30]:
# 3
ground_truth = ['Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(3, ground_truth)

{'X': array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (3196, 37)
	meta-types:[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE:

1 [0. 0.]
{'params': {'p': array([0.46217105, 0.53782895])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0EFCFD0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0EFC780>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EFC2E8>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.44490132, 0.55509868])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0EF8208>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0EF8320>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF8B70>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.58634868, 0.41365132])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0EFAA90>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteri

{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F028D0>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.15131579, 0.84868421])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0F025C0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F02240>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F02668>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.05098684, 0.94901316])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0F02550>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F02898>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F020F0>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.37417763, 0.62582237])}, 'prior': <bayesian.spn.a

{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0F0AF60>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0AC50>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.04141414, 0.95858586])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4AF07B8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4AF0748>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4AF0F98>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.65757576, 0.34242424])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4ACACF8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4ACAD68>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4ACAA5

{'params': {'p': array([1.00000000e+00, 1.73611111e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4B21908>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4B21438>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B216D8>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([1.73611111e-10, 1.00000000e+00])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4B214E0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4B21358>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B216A0>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([1.0000e+00, 1.5625e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4B21BA8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.P

(0.3804755944931164*((TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V8|p=[1.00000000e+00 8.22368421e-11])) + 0.3333333333333333*(Geometric(V8|p=0.5)) + 0.3333333333333333*(Poisson(V8|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V24|p=[8.22368421e-11 1.00000000e+00])) + 0.3333333333333333*(Geometric(V24|p=0.5)) + 0.3333333333333333*(Poisson(V24|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V27|p=[8.22368421e-11 1.00000000e+00])) + 0.3333333333333333*(Geometric(V27|p=0.5)) + 0.3333333333333333*(Poisson(V27|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V0|p=[0.13569079 0.86430921])) + 0.3333333333333333*(Geometric(V0|p=0.5)) + 0.3333333333333333*(Poisson(V0|mean=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V1|p=[0.04440789 0.95559211])) + 0.3333333333333333*(Geometric(V1|p=0.5)) + 0.3333333333333333*(Poisson(V1|mean=2))) * TypeMixtureUnconst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Done iteration 1/1 in 0.4463313345518145


------------------
Done in 0.45796834845906176
Dumped id to ./exp/temp/data.mat\20181105-195007\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195007\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195007
['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.COUNT']


In [31]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.08108108108108109


In [32]:
# 1510
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1510, ground_truth)

{'X': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 4.601e-01, 1.189e-01,
        1.000e+00],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 2.750e-01, 8.902e-02,
        1.000e+00],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 3.613e-01, 8.758e-02,
        1.000e+00],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 2.218e-01, 7.820e-02,
        1.000e+00],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 4.087e-01, 1.240e-01,
        1.000e+00],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 2.871e-01, 7.039e-02,
        0.000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (569, 31)
	meta-types:[1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 4]
	maximal discrete cardinality: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

{'params': {'alpha': 97.56377531321496, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EFACF8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EFA208>}
gamma fit params (7.222828593197023, 0, 0.03887362290798865)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EFA278>}
{'params': {'alpha': 7.222828593197023, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EFAE10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EFAC50>}
gamma fit params (5.029810552119686, 0, 0.24296149580089463)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EFA710>}
{'params': {'alpha': 5.029810552119686, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obje

{'params': {'alpha': 38.87744865028483, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4C2D780>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4C2DBA8>}
gamma fit params (39.19579106026804, 0, 0.0019930281137231793)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4C2DEF0>}
{'params': {'alpha': 39.19579106026804, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4C2D898>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4C2DD30>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.96449704, 0.03550296])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE4C2DF60>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE4C2DEB8>}
{'params': {'mean': 2}, 'p

gamma fit params (3.854201081940556, 0, 0.011752925612901517)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0A860>}
{'params': {'alpha': 3.854201081940556, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0AA20>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0A1D0>}
gamma fit params (8.826045086835855, 0, 0.0017646257765708841)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0A390>}
{'params': {'alpha': 8.826045086835855, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0A198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0A048>}
gamma fit params (6.450509802794781, 0, 0.0032218179094421575)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <

(0.5940246045694201*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=45.03133463022588;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=21.22271320784946;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=43.44801587434889;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=11.183757726494209;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=48.67534939713482;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUnconstrained(RE

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Done iteration 1/1 in 0.19812332968842838


------------------
Done in 0.2080064424189061
Dumped id to ./exp/temp/data.mat\20181105-195013\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195013\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195013
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [33]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [34]:
# 1489
ground_truth = ['Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1489, ground_truth)

{'X': array([[ 0.489927, -0.451528, -1.04799 , -0.598693, -0.020418,  0.      ],
       [-0.641265,  0.109245,  0.29213 , -0.916804,  0.240223,  0.      ],
       [ 0.870593, -0.459862,  0.578159,  0.806634,  0.835248,  0.      ],
       ...,
       [ 0.246882, -0.793228,  1.190101,  1.423194, -1.303036,  1.      ],
       [-0.778907, -0.383111,  1.727029, -1.432389, -1.208085,  0.      ],
       [-0.794604, -0.640053,  0.632221,  0.72028 , -1.231182,  1.      ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 2, 4]), 'R': array([1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (5404, 6)
	meta-types:[2 2 2 2 2 4]
	maximal discrete cardinality: [1 1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([-2.933743 ,  3.8285749]), array([-3.03870893,  3.66476297]), array([-2.79586911,  2.63086796]), array([-2.48534012,  3.04694796]), array([-2.36746097,  4.58666611]),

(0.5046262028127313*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V5|p=[0.49908324 0.50091676])) 

In [35]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5


In [36]:
# 37
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(37, ground_truth)

{'X': array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 2, 2, 1, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (768, 9)
	meta-types:[2 2 2 2 2 2 1 1 4]
	maximal discrete cardinality: [1 1 1 1 1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 0., 17.]), array([  0., 199.]), array([  0., 122.]), array([ 0., 99.]), array([  0., 846.]), array([ 0.        , 67.099

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF6F60>}
gamma fit params (2.5109224726262864, 0, 0.20267432243967715)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EF6128>}
{'params': {'alpha': 2.5109224726262864, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B05CC0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B05898>}
gamma fit params (12.676820593720754, 0, 3.159232183792262)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B05E10>}
{'params': {'alpha': 12.676820593720754, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B05B38>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B059E8>}




In [37]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [38]:
# 1479
ground_truth = []
for i in range(100):
    ground_truth.append('Type.POSITIVE')
ground_truth.append('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008ED99DA278>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008ED99DA908>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EDCF88160>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'alpha': 1.7917450699943174, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A7A320>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A7A6A0>}
gamma fit params (1.7806787234431918, 0, 58.509740076549804)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A7A4E0>}
{'params': {'alpha': 1.7806787234431918, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A7A5F8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A7AF28>}
gamma fit params (1.8087179444833978, 0, 57.68304631811555)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A7A978>}
{'params': {'alpha': 1.8087179444833978, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma ob

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE251DA20>}
gamma fit params (1.7776011109010725, 0, 58.92994406822095)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE251D400>}
{'params': {'alpha': 1.7776011109010725, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE251D4A8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE251D978>}
gamma fit params (1.7646087863499302, 0, 59.71978961279681)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE251D4E0>}
{'params': {'alpha': 1.7646087863499302, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE251DC88>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE251D550>}
gamm

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0E6F828>}
{'params': {'alpha': 1.762958122630515, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0E6F6A0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0E6F9B0>}
gamma fit params (1.7776716177560679, 0, 58.647022644643336)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0E6FB38>}
{'params': {'alpha': 1.7776716177560679, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0E6F198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0E6FE80>}
gamma fit params (1.804586229637459, 0, 57.63433705304342)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

(0.5*(((0.5066006600660066*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.6545988685199116;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.6570433180904642;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.6414960718966134;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.656301074955083;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.6658529706781784;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUncons

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.4680302856849607


------------------
Done in 1.4931628250322717
Dumped id to ./exp/temp/data.mat\20181105-195026\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195026\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195026
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITI

In [39]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9900990099009901


In [40]:
# 1487
ground_truth = []
for i in range(73):
    ground_truth.append('Type.POSITIVE')
for i in [54, 55, 60, 62, 65, 70]:
    ground_truth[i] = 'Type.REAL'
ground_truth[72] = ('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008ED99DA278>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008ED99DA908>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EDCF88160>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'alpha': 1.7917450699943174, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE107E860>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE107EC18>}
gamma fit params (1.7806787234431918, 0, 58.509740076549804)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE107E550>}
{'params': {'alpha': 1.7806787234431918, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE107E9B0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE107E438>}
gamma fit params (1.8087179444833978, 0, 57.68304631811555)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE107E898>}
{'params': {'alpha': 1.8087179444833978, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma ob

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B33A58>}
gamma fit params (1.7776011109010725, 0, 58.92994406822095)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0B33518>}
{'params': {'alpha': 1.7776011109010725, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B336D8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B33898>}
gamma fit params (1.7646087863499302, 0, 59.71978961279681)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0B334E0>}
{'params': {'alpha': 1.7646087863499302, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B33048>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B333C8>}
gamm

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0B09F60>}
{'params': {'alpha': 1.762958122630515, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B09860>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B09400>}
gamma fit params (1.7776716177560679, 0, 58.647022644643336)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0B09080>}
{'params': {'alpha': 1.7776716177560679, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B092E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0B09470>}
gamma fit params (1.804586229637459, 0, 57.63433705304342)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

(0.5*(((0.5066006600660066*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.6545988685199116;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.6570433180904642;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.6414960718966134;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.656301074955083;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.6658529706781784;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUncons

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.5461919856624604


------------------
Done in 1.5932157223443824
Dumped id to ./exp/temp/data.mat\20181105-195430\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195430\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195430
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITI

In [41]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9041095890410958


In [42]:
# 1063
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1063, ground_truth)

{'X': array([[  1.1,   1.4,   1.4, ...,   1.2,   1.4,   0. ],
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [415. ,  59. ,  50. , ..., 467. , 106. ,   1. ],
       ...,
       [  4. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [  4. ,   1. ,   1. , ...,   2. ,   1. ,   1. ],
       [  3. ,   1. ,   1. , ...,   0. ,   1. ,   1. ]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 1, 2, 1, 2, 1, 4]), 'R': array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 12,  1,
        1,  1,  1,  1,  2])}

Loaded bayesian/data/temp/data.mat
	with shape: (522, 22)
	meta-types:[1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 4 1 2 1 2 1 4]
	maximal discrete cardinality: [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  1  1  1  1  1  2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1>

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C51D30>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51828>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51BE0>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C51A20>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51EB8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0FEACF8>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0FEA1D0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0FEA5F8>}
{

gamma fit params (0.47915161206174856, 0, 85523.99600922647)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EF6940>}
{'params': {'alpha': 0.47915161206174856, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF6438>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF60B8>}
gamma fit params (1.032372226696916, 0, 0.38399476790303255)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EF6320>}
{'params': {'alpha': 1.032372226696916, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF6EB8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF6A90>}
gamma fit params (0.4612526895107892, 0, 5247.000962016665)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <baye

 4.25531913e-10 4.25531913e-10 4.25531955e-03 4.25531955e-03])) + 0.3333333333333333*(Geometric(V15|p=0.5)) + 0.3333333333333333*(Poisson(V15|mean=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V16|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V16|alpha=10.602515921394968;beta=1)) + 0.3333333333333333*(Exponential(V16|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V17|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V17|alpha=2.515596398219457;beta=1)) + 0.3333333333333333*(Exponential(V17|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V18|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V18|alpha=1.369192707680947;beta=1)) + 0.3333333333333333*(Exponential(V18|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V19|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V19|alpha=1.350406566610991;beta=1)) + 0.3333333333333333*(Exponential(V19|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V20|mean=0;st

In [43]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9090909090909091


In [44]:
# 1471
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1471, ground_truth)

{'X': array([[4.32923e+03, 4.00923e+03, 4.28923e+03, ..., 4.63590e+03,
        4.39385e+03, 0.00000e+00],
       [4.32462e+03, 4.00462e+03, 4.29385e+03, ..., 4.63282e+03,
        4.38410e+03, 0.00000e+00],
       [4.32769e+03, 4.00667e+03, 4.29538e+03, ..., 4.62872e+03,
        4.38923e+03, 0.00000e+00],
       ...,
       [4.27744e+03, 3.99077e+03, 4.24667e+03, ..., 4.59179e+03,
        4.33949e+03, 1.00000e+00],
       [4.28462e+03, 3.99179e+03, 4.25128e+03, ..., 4.59641e+03,
        4.35077e+03, 1.00000e+00],
       [4.28769e+03, 3.99744e+03, 4.26000e+03, ..., 4.59795e+03,
        4.35077e+03, 1.00000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (14980, 15)
	meta-types:[1 1 1 1 1 1 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaTyp

gamma fit params (49609.52668709937, 0, 0.08444904537435581)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C51828>}
{'params': {'alpha': 49609.52668709937, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51D30>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51470>}
gamma fit params (23251.604877801612, 0, 0.18354400776026863)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C517B8>}
{'params': {'alpha': 23251.604877801612, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51A58>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51908>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.58530864, 0.41469136])}, 'prior': <bayesian.spn.algori

{'params': {'alpha': 148672.49766605912, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE09C9438>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE09C9A58>}
gamma fit params (92101.31913718744, 0, 0.046521392761223856)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE09C9D30>}
{'params': {'alpha': 92101.31913718744, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE267D358>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE267DEB8>}
gamma fit params (39549.02588900911, 0, 0.10464314231751574)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE267DA90>}
{'params': {'alpha': 39549.02588900911, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma ob

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D2E8>}
gamma fit params (20849.46511594375, 0, 0.22315577163661912)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0DB00>}
{'params': {'alpha': 20849.46511594375, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D160>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D358>}
gamma fit params (18355.8262405481, 0, 0.24032128307725786)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0D940>}
{'params': {'alpha': 18355.8262405481, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D898>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0DA90>}
gamma 

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F273C8>}
{'params': {'alpha': 419.607399232851, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F27BE0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F27DD8>}
gamma fit params (126253.69242283086, 0, 0.033411191760719015)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F27780>}
{'params': {'alpha': 126253.69242283086, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F27A20>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F272B0>}
gamma fit params (465577.54694444797, 0, 0.00922386250522783)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma obje

{'params': {'alpha': 69118.49054776171, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6B70>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6C88>}
gamma fit params (251851.36639907776, 0, 0.017096728562671235)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE28C6630>}
{'params': {'alpha': 251851.36639907776, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C68D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6320>}
gamma fit params (51448.75700017074, 0, 0.0905572949020771)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE28C6BE0>}
{'params': {'alpha': 51448.75700017074, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma ob

(0.45927903871829107*(((0.2719476744186046*(((0.6002137894174239*(((0.5627782724844167*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=223867.17478848403;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V13|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V13|alpha=222339.08954044903;beta=1)) + 0.3333333333333333*(Exponential(V13|l=2))))) + 0.43722172751558325*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=23829.14019707002;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V13|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V13|alpha=52528.49717230226;beta=1)) + 0.3333333333333333*(Exponential(V13|l=2)))))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V10|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V10|alp

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Done iteration 1/1 in 1.5790937190363366


------------------
Done in 1.5924747652188103
Dumped id to ./exp/temp/data.mat\20181105-195551\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195551\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195551
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [45]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [46]:
# 1467
ground_truth = ['Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1467, ground_truth)

{'X': array([[1.00000e+00, 1.00000e+00, 5.02000e+02, ..., 7.96997e-01,
        8.69893e-01, 0.00000e+00],
       [1.00000e+00, 2.00000e+00, 2.49000e+02, ..., 4.38447e-01,
        5.12256e-01, 1.00000e+00],
       [1.00000e+00, 3.00000e+00, 2.02000e+02, ..., 2.85636e-01,
        3.65858e-01, 1.00000e+00],
       ...,
       [3.00000e+00, 1.78000e+02, 5.09000e+02, ..., 4.61632e-01,
        6.52817e-01, 1.00000e+00],
       [3.00000e+00, 1.79000e+02, 4.21000e+02, ..., 2.01469e-01,
        5.36535e-01, 1.00000e+00],
       [3.00000e+00, 1.80000e+02, 1.80000e+01, ..., 7.60344e-01,
        7.62439e-01, 1.00000e+00]], dtype=float32), 'T': array([4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (540, 21)
	meta-types:[4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
	meta types [<MetaTyp

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EF6A90>}
{'params': {'alpha': 2.0272453372474493, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF69E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF6198>}
gamma fit params (24.494136337807813, 0, 0.030562400751064107)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0EF6470>}
{'params': {'alpha': 24.494136337807813, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF68D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0EF60B8>}
gamma fit params (1.7118501696010877, 0, 0.2815923199114436)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma obj

{'params': {'alpha': 1.843151971118298, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6E48>}
gamma fit params (1.6975806756815652, 0, 0.2869372494408129)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE28C61D0>}
{'params': {'alpha': 1.6975806756815652, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6FDA0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6F2E8>}
gamma fit params (1.616463489348701, 0, 0.30389925593773454)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0A6FE10>}
{'params': {'alpha': 1.616463489348701, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obj

(0.4981481481481482*((TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V0|p=[3.34572491e-01 3.34572491e-01 3.30855018e-01 3.71747211e-10])) + 0.3333333333333333*(Geometric(V0|p=0.5)) + 0.3333333333333333*(Poisson(V0|mean=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=2.0627746971922356;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.7299185802656771;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.702845815873681;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.700515105893134;beta=1)) + 0.3333333333333333*(Exponential(V4|l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Done iteration 1/1 in 0.13320490602831114


------------------
Done in 0.13829054835559873
Dumped id to ./exp/temp/data.mat\20181105-195611\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195611\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195611
['Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT']


In [47]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9047619047619048


In [48]:
# 1480
ground_truth = ['Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1480, ground_truth)

{'X': array([[65.  ,  0.  ,  0.7 , ...,  3.3 ,  0.9 ,  0.  ],
       [62.  ,  1.  , 10.9 , ...,  3.2 ,  0.74,  0.  ],
       [62.  ,  1.  ,  7.3 , ...,  3.3 ,  0.89,  0.  ],
       ...,
       [52.  ,  1.  ,  0.8 , ...,  3.2 ,  1.  ,  0.  ],
       [31.  ,  1.  ,  1.3 , ...,  3.4 ,  1.  ,  0.  ],
       [38.  ,  1.  ,  1.  , ...,  4.4 ,  1.5 ,  1.  ]], dtype=float32), 'T': array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (583, 11)
	meta-types:[1 4 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [1 2 1 1 1 1 1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.DISCRETE: 3> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 4., 90.]), array([0, 1], dtype=int64), array([ 0.40000001, 75.        ]), array([ 0.1       , 19.70000076]), array([  63., 2110.]), a

{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A66240>}
gamma fit params (0.947893624336071, 0, 6.704971769895008)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A66FD0>}
{'params': {'alpha': 0.947893624336071, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A66E48>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A669B0>}
gamma fit params (0.814883658414526, 0, 3.706050510397895)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A667B8>}
{'params': {'alpha': 0.814883658414526, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A66630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A663C8>}
gamma

(0.5711835334476844*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=5.572847386934386;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V1|p=[0.31231231 0.68768769])) + 0.3333333333333333*(Geometric(V1|p=0.5)) + 0.3333333333333333*(Poisson(V1|mean=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=4.296540988663734;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.6719399856055104;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=6.960885501856301;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUnconstrained(REAL, 

In [49]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9090909090909091


In [50]:
# 1068
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1068, ground_truth)

{'X': array([[  1.1,   1.4,   1.4, ...,   1.2,   1.4,   0. ],
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [ 91. ,   9. ,   3. , ..., 126. ,  17. ,   1. ],
       ...,
       [  5. ,   3. ,   3. , ...,   6. ,   5. ,   0. ],
       [ 18. ,   8. ,   5. , ...,  50. ,  15. ,   0. ],
       [ 26. ,  18. ,  13. , ..., 109. ,  35. ,   0. ]], dtype=float32), 'T': array([2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 4]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (1109, 22)
	meta-types:[2 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 1 4]
	maximal discrete cardinality: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


gamma fit params (5.9188847553123916, 0, 0.23112699710603687)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE28C6E48>}
{'params': {'alpha': 5.9188847553123916, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C61D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6B00>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE28C6320>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6A90>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE28C6BE0>}




CATEGORICAL




1 [0. 0.]
{'params': {'p': array([0.96660808, 0.03339192])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE2A66FD0>}
{'params': {'

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0DAC8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6FCF8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6FC18>}
gamma fit params (3.901497754357835, 0, 1.3433161344828521)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0A6FAC8>}
{'params': {'alpha': 3.901497754357835, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6F780>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0A6FE80>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0A6F5C0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <baye

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51CF8>}
gamma fit params (2402.765407721859, 0, 0.0004167347283165016)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C51E48>}
{'params': {'alpha': 2402.765407721859, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C516D8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C518D0>}
gamma fit params (2402.765407721859, 0, 0.0004167347283165016)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0C517B8>}
{'params': {'alpha': 2402.765407721859, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51518>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51128>}


{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE11C0710>}
{'params': {'alpha': 12.335917850215889, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F17128>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F17358>}
gamma fit params (9.852330832268756, 0, 4.082469791988384)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F17C50>}
{'params': {'alpha': 9.852330832268756, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F17630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F17F28>}
gamma fit params (2.699167943185453, 0, 3.1042305506026815)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at

(0.5130748422001803*(((0.44639718804920914*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.9928284231137754;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V5|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V5|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V5|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V6|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V6|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V6|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V7|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V7|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V7|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Done iteration 1/1 in 0.24910370636462176


------------------
Done in 0.25607509839295517
Dumped id to ./exp/temp/data.mat\20181105-195613\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195613\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195613
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [51]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [52]:
# 1492
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']

accuracy = evaluate_bayesian(1492, ground_truth)

{'X': array([[5.79e-04, 6.09e-04, 5.51e-04, ..., 6.17e-04, 6.14e-04, 1.20e+01],
       [6.30e-04, 6.61e-04, 7.19e-04, ..., 6.71e-04, 6.67e-04, 1.20e+01],
       [6.16e-04, 6.15e-04, 6.06e-04, ..., 6.07e-04, 6.02e-04, 1.20e+01],
       ...,
       [7.01e-04, 6.42e-04, 6.46e-04, ..., 6.22e-04, 6.63e-04, 2.00e+00],
       [7.09e-04, 7.76e-04, 7.16e-04, ..., 6.83e-04, 7.02e-04, 2.00e+00],
       [8.50e-04, 7.70e-04, 7.64e-04, ..., 8.10e-04, 7.97e-04, 2.00e+00]],
      dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

Loaded bayesian/data/temp/data.mat
	with shape: (1600, 65)
	meta-typ

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4224470>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4224EB8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4224D68>}


D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE245B748>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE245B198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE245B668>}
gamma fit params (12.849018071847684, 0, 3.3961159326277766e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE47D17F0>}
{'params': {'alpha': 12.849018071847684, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE47D1630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE47D1BA8>}
gamma fit params (12.833858922022932, 0, 3.213096235463534e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0

...

gamma fit params (43.58490522605466, 0, 1.71685740514581e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B02B70>}
{'params': {'alpha': 43.58490522605466, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B02F98>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B029B0>}
gamma fit params (40.95187465849605, 0, 1.9428391857701196e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B02EF0>}
{'params': {'alpha': 40.95187465849605, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B02748>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B02390>}
gamma fit params (40.03240551367586, 0, 2.1108929230193512e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior':

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE3F6B5C0>}
{'params': {'alpha': 34.393381802419846, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F6BA58>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F6B898>}
gamma fit params (26.060813696877013, 0, 1.7970398641222116e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE3F6BEF0>}
{'params': {'alpha': 26.060813696877013, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F6BFD0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE3F6B630>}
gamma fit params (21.93591314803734, 0, 2.0669261829378848e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamm

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0F198>}
{'params': {'alpha': 29.747838486363104, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0FF60>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0FC88>}
gamma fit params (28.78419871880063, 0, 3.3725270255702386e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0F0F0F0>}
{'params': {'alpha': 28.78419871880063, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0F518>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0F0FBE0>}
True True True
True True True


(0.436875*(((0.43204577968526464*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=12.849018071847684;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=12.833858922022932;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=12.540056076017919;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=11.867288992163187;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=11.441125187891954;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtur

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
Done iteration 1/1 in 1.0997516984379558


------------------
Done in 1.1230659913706518
Dumped id to ./exp/temp/data.mat\20181105-195617\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195617\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195617
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Typ

In [53]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9846153846153847


In [54]:
# 1050
ground_truth = ['Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 
                'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']

accuracy = evaluate_bayesian(1050, ground_truth)

{'X': array([[ 2.  ,  1.  ,  0.  , ...,  0.  , 10.  ,  0.  ],
       [ 1.  ,  1.  ,  4.  , ...,  0.  , 14.  ,  0.  ],
       [27.  , 19.  ,  1.  , ..., 30.91, 42.  ,  0.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  7.  ,  0.  , ...,  0.  , 11.  ,  0.  ],
       [ 1.  ,  9.  ,  0.  , ...,  0.  , 11.  ,  0.  ]], dtype=float32), 'T': array([2, 1, 4, 4, 2, 2, 1, 1, 2, 2, 1, 1, 1, 4, 2, 2, 4, 2, 2, 2, 2, 1,
       2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 4]), 'R': array([ 1,  1, 25, 43,  1,  1,  1,  1,  1,  1,  1,  1,  1, 71,  1,  1,  8,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  2])}

Loaded bayesian/data/temp/data.mat
	with shape: (1563, 38)
	meta-types:[2 1 4 4 2 2 1 1 2 2 1 1 1 4 2 2 4 2 2 2 2 1 2 2 2 1 2 2 1 1 2 1 2 1 1 2 2
 4]
	maximal discrete cardinality: [ 1  1 25 43  1  1  1  1  1  1  1  1  1 71  1  1  8  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  2]
	meta types [<M

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)






CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
{'params': {'p': array([3.81258022e-01, 3.20924261e-01, 1.55327342e-01, 7.57381257e-02,
       2.82413351e-02, 1.54043646e-02, 1.15532735e-02, 3.85109126e-03,
       2.56739422e-03, 3.85109126e-03, 1.28369717e-03, 1.28369704e-10,
       1.28369704e-10, 1.28369704e-10, 1.28369704e-10, 1.28369704e-10,
       1.28369704e-10, 1.28369704e-10, 1.28369704e-10, 1.28369704e-10,
       1.28369704e-10, 1.28369704e-10, 1.28369704e-10, 1.28369704e-10,
       1.28369704e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000008EE0C51588>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000008EE0C51C18>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0C51208>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 

{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2776BE0>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0DF28>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D4A8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D780>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE4B0D1D0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0D8D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE4B0DE48>}
gamma fit params (3.101436400507435, 0, 0.13911796532583917)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNorm

{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A25780>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A25898>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A25668>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A252E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A25208>}
gamma fit params (3.1459869462838497, 0, 0.06917072910888865)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2A25320>}
{'params': {'alpha': 3.1459869462838497, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2A253C8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Poste

...

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0AE0588>}
gamma fit params (8.287266953247467, 0, 1.7353750843083617)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0AE0C50>}
{'params': {'alpha': 8.287266953247467, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0AE0128>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0AE0B00>}
gamma fit params (5.369812317045459, 0, 7.157795596568256)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0AE0470>}
{'params': {'alpha': 5.369812317045459, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0AE0860>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0AE07B8>}
gamma f

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0BDD780>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0BDD2E8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0BDD320>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0BDD6D8>}
gamma fit params (0.5451401513079825, 0, 13770.383439788746)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE0BDDBE0>}
{'params': {'alpha': 0.5451401513079825, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0BDDF98>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE0BDD0F0>}
gamma fit params (1.2004086616644662, 0, 2136.9151396388015)
{'params': {'mean

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([8.64864851e-01, 2.25225222e-10, 6.75675667e-02, 2.02702702e-02,
       1.80180180e-02, 6.75675687e-03, 2.25225244e-03, 2.25225244e-03,
       2.25225244e-03, 2.25225244e-03, 2.25225244e-03, 2.25225222e-10,
       2.25225244e-03, 2.25225222e-10, 4.50450466e-03, 2.25225222e-10,
       2.25225222e-10, 2.25225244e-03, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225244e-03, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.25225222e-10,
       2.25225222e-10, 2.25225222e-10, 2.25225222e-10, 2.252

{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE29669E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2966710>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE29661D0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE29668D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2966D68>}
gamma fit params (4.2630674694405375, 0, 0.8071386916048394)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000008EE2966278>}
{'params': {'alpha': 4.2630674694405375, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000008EE2966E48>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Poster

 0.00255102 0.00255102])) + 0.3333333333333333*(Geometric(V16|p=0.5)) + 0.3333333333333333*(Poisson(V16|mean=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V33|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V33|alpha=9.902003983851777;beta=1)) + 0.3333333333333333*(Exponential(V33|l=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V37|p=[0.82397959 0.17602041])) + 0.3333333333333333*(Geometric(V37|p=0.5)) + 0.3333333333333333*(Poisson(V37|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
Done iteration 1/1 in 0.5523880767879064


------------------
Done in 0.5655859561584293
Dumped id to ./exp/temp/data.mat\20181105-195642\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181105-195642\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181105-195642
['Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.CATEGORICAL', 'Typ

In [55]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.6842105263157895


In [56]:
# 1462
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.REAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1462, ground_truth)

{'X': array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699,   0.     ],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ,   0.     ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645,   0.     ],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ,   1.     ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ,   1.     ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ,   1.     ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 4]), 'R': array([1, 1, 1, 1, 2])}

Loaded bayesian/data/temp/data.mat
	with shape: (1372, 5)
	meta-types:[2 2 2 2 4]
	maximal discrete cardinality: [1 1 1 1 2]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([-7.04209995,  6.82480001]), array([-13.7730999 ,  12.95160007]), array([-5.28609991, 17.92740059]), array([-8.54819965,  2.44950008]), array([0, 1], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <M

(0.48177842565597667*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=3.977255468506035;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(DISCRETE, 0.3333333333333333*(Categorical(V4|p=[0.62481089 0.37518911])) + 0.3333333333333333*(Geometric(V4|p=0.5)) + 0.3333333333333333*(Poisson(V4|mean=2))))) + 0.5182215743440233*(((0.5246132208157525*((TypeMixtureUnconstrained(REA

In [57]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.8


The results are summarized as follows.

In [62]:
bayesian_acc = {'Dataset_id': [31, 1464, 334, 50, 333, 1494, 3, 1510, 1489, 37,
                                   1479, 1487, 1063, 1471, 1467, 1480, 1068, 1492, 1050, 1462],
                'Accuracy': [0.619, 1, 0.857, 0, 0.857, 0.595, 0.081, 1, 0.5, 1, 0.99, 0.904, 0.955, 1, 0.905, 0.909 ,1, 0.984, 0.684, 0.8]}
df_bayesian_acc = pd.DataFrame(bayesian_acc) 

In [63]:
df_bayesian_acc[['Dataset_id', 'Accuracy']]

,Dataset_id,Accuracy
0,31,0.619
1,1464,1.000
2,334,0.857
3,50,0.000
4,333,0.857
5,1494,0.595
6,3,0.081
7,1510,1.000
8,1489,0.500
9,37,1.000
